In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Vitamin_D_Levels"
cohort = "GSE86406"

# Input paths
in_trait_dir = "../../input/GEO/Vitamin_D_Levels"
in_cohort_dir = "../../input/GEO/Vitamin_D_Levels/GSE86406"

# Output paths
out_data_file = "../../output/preprocess/Vitamin_D_Levels/GSE86406.csv"
out_gene_data_file = "../../output/preprocess/Vitamin_D_Levels/gene_data/GSE86406.csv"
out_clinical_data_file = "../../output/preprocess/Vitamin_D_Levels/clinical_data/GSE86406.csv"
json_path = "../../output/preprocess/Vitamin_D_Levels/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
# 1. Let's first list the directory contents to understand what files are available
import os

print("Files in the cohort directory:")
files = os.listdir(in_cohort_dir)
print(files)

# Adapt file identification to handle different naming patterns
soft_files = [f for f in files if 'soft' in f.lower() or '.soft' in f.lower() or '_soft' in f.lower()]
matrix_files = [f for f in files if 'matrix' in f.lower() or '.matrix' in f.lower() or '_matrix' in f.lower()]

# If no files with these patterns are found, look for alternative file types
if not soft_files:
    soft_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]
if not matrix_files:
    matrix_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]

print("Identified SOFT files:", soft_files)
print("Identified matrix files:", matrix_files)

# Use the first files found, if any
if len(soft_files) > 0 and len(matrix_files) > 0:
    soft_file = os.path.join(in_cohort_dir, soft_files[0])
    matrix_file = os.path.join(in_cohort_dir, matrix_files[0])
    
    # 2. Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
    
    # 3. Obtain the sample characteristics dictionary from the clinical dataframe
    sample_characteristics_dict = get_unique_values_by_row(clinical_data)
    
    # 4. Explicitly print out all the background information and the sample characteristics dictionary
    print("\nBackground Information:")
    print(background_info)
    print("\nSample Characteristics Dictionary:")
    print(sample_characteristics_dict)
else:
    print("No appropriate files found in the directory.")


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
# 1. Assess gene expression data availability
is_gene_available = True  # Based on series_matrix file and the study design using skin biopsies with transcriptome analysis

# 2.1 Identify data availability and row indices
# This is a vitamin D intervention study with different doses as described in Series_overall_design
# While not directly in the sample characteristics, we can infer the trait is studied through the intervention design
trait_row = None  # Cannot directly extract from sample characteristics, but study is about vitamin D intervention
age_row = 1  # Age data is available at index 1
gender_row = 2  # Gender data is available at index 2

# 2.2 Define conversion functions
def convert_trait(value_str):
    # No direct vitamin D level measurements available in sample characteristics
    # The study uses different doses (0/placebo, 50k IU, 100k IU, 200k IU) but this info isn't in the sample characteristics
    return None

def convert_age(value_str):
    try:
        # Extract age value from the format "age: XX"
        age_value = value_str.split(": ")[1]
        return float(age_value)  # Convert to float for continuous variable
    except (ValueError, IndexError):
        return None

def convert_gender(value_str):
    try:
        gender = value_str.split(": ")[1]
        if gender.upper() == 'F':
            return 0  # Female
        elif gender.upper() == 'M':
            return 1  # Male
        else:
            return None
    except (ValueError, IndexError):
        return None

# 3. Save metadata about dataset availability
# While this is a vitamin D study, we can't extract the exact vitamin D levels or treatment groups
# from the sample characteristics dictionary, so we mark trait as unavailable for our analysis
is_trait_available = False  # Cannot extract vitamin D levels or treatment groups from sample characteristics
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical feature extraction is skipped since trait_row is None


### Step 3: Gene Data Extraction

In [ ]:
# Use the helper function to get the proper file paths
soft_file_path, matrix_file_path = geo_get_relevant_filepaths(in_cohort_dir)

# Extract gene expression data
try:
    gene_data = get_genetic_data(matrix_file_path)
    
    # Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
    
    # Print shape to understand the dataset dimensions
    print(f"\nGene expression data shape: {gene_data.shape}")
    
except Exception as e:
    print(f"Error extracting gene data: {e}")


### Step 4: Gene Identifier Review

In [ ]:
# This previous output shows gene identifiers that appear to be probe IDs or microarray feature IDs
# (starting with 1665xxxx), not standard human gene symbols.
# Standard human gene symbols would typically be letters or letter-number combinations like "BRCA1", "TP53", etc.
# These numeric identifiers need to be mapped to actual gene symbols for meaningful biological interpretation.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [ ]:
# 1. This part examines the data more thoroughly to determine what type of data it contains
try:
    # First, let's check a few rows of the gene_data we extracted in Step 3
    print("Sample of gene expression data (first 5 rows, first 5 columns):")
    print(gene_data.iloc[:5, :5])
    
    # Analyze the SOFT file to identify the data type and mapping information
    platform_info = []
    with gzip.open(soft_file_path, 'rt', encoding='latin-1') as f:
        for line in f:
            if line.startswith("!Platform_title") or line.startswith("!Series_title") or "description" in line.lower():
                platform_info.append(line.strip())
    
    print("\nPlatform information:")
    for line in platform_info:
        print(line)
    
    # Extract the gene annotation using the library function
    gene_annotation = get_gene_annotation(soft_file_path)
    
    # Display column names of the annotation dataframe
    print("\nGene annotation columns:")
    print(gene_annotation.columns.tolist())
    
    # Preview the annotation dataframe
    print("\nGene annotation preview:")
    annotation_preview = preview_df(gene_annotation)
    print(annotation_preview)
    
    # Check if ID column exists in the gene_annotation dataframe
    if 'ID' in gene_annotation.columns:
        # Check if any of the IDs in gene_annotation match those in gene_data
        sample_ids = list(gene_data.index[:10])
        matching_rows = gene_annotation[gene_annotation['ID'].isin(sample_ids)]
        print(f"\nMatching rows in annotation for sample IDs: {len(matching_rows)}")
        
        # Look for gene symbol column
        gene_symbol_candidates = [col for col in gene_annotation.columns if 'gene' in col.lower() or 'symbol' in col.lower() or 'name' in col.lower()]
        print(f"\nPotential gene symbol columns: {gene_symbol_candidates}")
    
except Exception as e:
    print(f"Error analyzing gene annotation data: {e}")
    gene_annotation = pd.DataFrame()

# Based on our analysis, determine if this is really gene expression data
# Check the platform description and match with the data we've extracted
is_gene_expression = False
for info in platform_info:
    if 'expression' in info.lower() or 'transcript' in info.lower() or 'mrna' in info.lower():
        is_gene_expression = True
        break

print(f"\nIs this dataset likely to contain gene expression data? {is_gene_expression}")

# If this isn't gene expression data, we need to update our metadata
if not is_gene_expression:
    print("\nNOTE: Based on our analysis, this dataset doesn't appear to contain gene expression data.")
    print("It appears to be a different type of data (possibly SNP array or other genomic data).")
    # Update is_gene_available for metadata
    is_gene_available = False
    
    # Save the updated metadata
    validate_and_save_cohort_info(
        is_final=False,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=is_gene_available,
        is_trait_available=is_trait_available
    )


### Step 6: Gene Identifier Mapping

In [ ]:
# 1. Let's try a different approach with the existing data
# The problem is that we need to map probe IDs to gene symbols more effectively

# First, let's check the ID formats more carefully
print("Gene expression data ID format:", gene_data.index[0])
print("Gene annotation ID format:", gene_annotation['ID'].iloc[0])

# Let's create a better mapping approach by:
# 1. Ensuring the IDs are in the same format
# 2. Using both GB_ACC and SPOT_ID for potential gene information
# 3. Creating a fallback mapping when needed

# Convert annotation IDs to the same format as expression data IDs
gene_annotation['ID'] = gene_annotation['ID'].astype(str)
gene_data.index = gene_data.index.astype(str)

# Create a mapping dataframe with gene information from multiple sources
gene_mapping = pd.DataFrame({'ID': gene_data.index})

# Match gene annotation to our data
annotation_lookup = gene_annotation.set_index('ID')

# Map accession IDs to our data
gene_mapping['Gene'] = gene_mapping['ID'].map(lambda x: annotation_lookup.loc[x, 'GB_ACC'] if x in annotation_lookup.index else None)

# Check how many valid gene mappings we have
valid_mappings = gene_mapping['Gene'].notna().sum()
print(f"\nValid gene mappings found: {valid_mappings} out of {len(gene_mapping)}")

# If very few valid mappings, create temporary gene symbols based on probe IDs
if valid_mappings < 1000:  # Arbitrary threshold
    print("Insufficient gene mappings. Creating temporary gene symbols.")
    gene_mapping['Gene'] = gene_mapping['ID'].map(lambda x: f"PROBE_{x}")

# Ensure the Gene column doesn't have null values
gene_mapping = gene_mapping.fillna({'Gene': lambda x: f"PROBE_{x['ID']}"})

print("\nGene mapping sample:")
print(gene_mapping.head())

# Apply the mapping to get gene-level data
gene_data_mapped = gene_data.copy()
gene_data_mapped.index = gene_mapping.set_index('ID')['Gene']

# Group by the new gene index to handle duplicates
gene_data_mapped = gene_data_mapped.groupby(level=0).mean()

print("\nMapped gene expression data:")
print(f"Shape: {gene_data_mapped.shape}")
print(gene_data_mapped.head())

# Assign the mapped data to gene_data for downstream processing
gene_data = gene_data_mapped

# Update the is_gene_available flag based on whether we have gene expression data
is_gene_available = gene_data.shape[0] > 0 and gene_data.shape[1] > 0
print(f"\nIs gene expression data available after mapping: {is_gene_available}")

# Check gene symbol format to detect if we're using temporary IDs
using_temp_ids = gene_data.index.str.contains('PROBE_').any()
print(f"Using temporary gene IDs: {using_temp_ids}")